# Classification

# 1. Loading Library

In [1]:
import pandas as pd
import numpy as np
import time
import os

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [4]:
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import fcluster
from sklearn.cluster import DBSCAN

# 2. Loading Data

## 2-1. client data

In [6]:
# 고객 데이터를 불러온다.
df_client = pd.read_csv("./data/01client_demo.csv", encoding="cp949")
df_client.head()

,고객번호,성별,연령대,거주지역
0,1,M,60세이상,60.0
1,2,M,60세이상,100.0
2,3,M,60세이상,33.0
3,4,F,60세이상,16.0
4,5,M,60세이상,100.0


In [7]:
# 서울시 데이터만 선별하기 위해 우편 데이터를 가져온다.
path = 'C:\\Users\\student\\Desktop\\aaa\\ModuleProject01\data\post_num'

file_list = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.xlsx' in file:
            file_list.append(os.path.join(r, file))

for f in file_list:
    print(f)

C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\강원도.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\경기도.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\경상남도.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\경상북도.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\광주시.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\대구시.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\대전시.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\부산시.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\서울시.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\세종시.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\울산시.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\인천시.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\전라남도.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01\data\post_num\전라북도.xlsx
C:\Users\student\Desktop\aaa\ModuleProject01

In [8]:
# 파일명을 가진 변수로 데이터프레임을 생성한다.
for file in file_list :
    save_file = file.split('\\')[-1].split('.')[0]
    globals()[save_file] = pd.read_excel(file.replace('\\', '\\\\'))

In [9]:
서울시.head()

,우편번호\n(구역번호),시도,시군구,읍면,도로명주소,범위종류
0,6267,서울특별시,강남구,NaN,강남대로 238 ~ 246,짝수
1,6266,서울특별시,강남구,NaN,강남대로 248 ~ 256,짝수
2,6265,서울특별시,강남구,NaN,강남대로 262 ~ 276,짝수
3,6258,서울특별시,강남구,NaN,강남대로 278 ~ 292,짝수
4,6253,서울특별시,강남구,NaN,강남대로 298 ~ 318,짝수


In [10]:
def s_cut_postnum(df):
    df['CutPostnum'] = df['우편번호\n(구역번호)'].apply(lambda x: str(x)[0:2])

In [11]:
s_cut_postnum(서울시)
서울시.head()

,우편번호\n(구역번호),시도,시군구,읍면,도로명주소,범위종류,CutPostnum
0,6267,서울특별시,강남구,NaN,강남대로 238 ~ 246,짝수,62
1,6266,서울특별시,강남구,NaN,강남대로 248 ~ 256,짝수,62
2,6265,서울특별시,강남구,NaN,강남대로 262 ~ 276,짝수,62
3,6258,서울특별시,강남구,NaN,강남대로 278 ~ 292,짝수,62
4,6253,서울특별시,강남구,NaN,강남대로 298 ~ 318,짝수,62


In [12]:
s_postnum = 서울시['CutPostnum']
s_koo = 서울시['시군구']

frame = { '거주지역': s_postnum, 'S_koo': s_koo } 

df_post_s = pd.DataFrame(frame)
df_post_s.head()

,거주지역,S_koo
0,62,강남구
1,62,강남구
2,62,강남구
3,62,강남구
4,62,강남구


In [13]:
df_post_s['거주지역'] = df_post_s['거주지역'].apply(lambda x : np.float64(x))
df_post_s = df_post_s.drop_duplicates()
s_df_client = df_client[df_client['거주지역']<100]
s_df_client.head()

,고객번호,성별,연령대,거주지역
0,1,M,60세이상,60.0
2,3,M,60세이상,33.0
3,4,F,60세이상,16.0
6,7,F,60세이상,36.0
7,8,M,60세이상,10.0


In [14]:
s_df_client = pd.merge(s_df_client, df_post_s, how='left')
s_df_client.head()

,고객번호,성별,연령대,거주지역,S_koo
0,1,M,60세이상,60.0,강남구
1,3,M,60세이상,33.0,은평구
2,4,F,60세이상,16.0,노원구
3,7,F,60세이상,36.0,서대문구
4,8,M,60세이상,10.0,강북구


In [38]:
len(s_df_client)

5841

## 2-2. purchase data

### 해당 데이터 크기가 커서 서울시 데이터로 불러와서 분석할 것!

In [16]:
# 고객 구매내역 불러오기 ... 시간이 오래걸려서 확인
start = time.time()

df_purchase = pd.read_csv('./data/02purchaseTR.csv', encoding='CP949')

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 18.661065578460693
  제휴사    영수증번호  대분류코드  중분류코드    소분류코드   고객번호  점포코드      구매일자  구매시간  구매금액
0   B  8664000     15   1504  B150401  17218    44  20140222    20  2420
1   B  8664000     16   1601  B160101  17218    44  20140222    20  1070
2   B  8664000     16   1602  B160201  17218    44  20140222    20  8060
3   B  8664000     18   1803  B180301  17218    44  20140222    20  6000
4   B  8664001      5    509  B050901  17674    44  20140222    22  1120


In [17]:
df_purchase.head()

,제휴사,영수증번호,대분류코드,중분류코드,소분류코드,고객번호,점포코드,구매일자,구매시간,구매금액
0,B,8664000,15,1504,B150401,17218,44,20140222,20,2420
1,B,8664000,16,1601,B160101,17218,44,20140222,20,1070
2,B,8664000,16,1602,B160201,17218,44,20140222,20,8060
3,B,8664000,18,1803,B180301,17218,44,20140222,20,6000
4,B,8664001,5,509,B050901,17674,44,20140222,22,1120


In [18]:
# 데이터 크기가 크기 때문에 sample 수 확인
df_purchase.shape

(28593030, 10)

In [20]:
# 서울시 고객 데이터의 구매내역만 불러온다.
s_df_purchase = pd.merge(s_df_client, df_purchase, how='left')
s_df_purchase.head()

,고객번호,성별,연령대,거주지역,S_koo,제휴사,영수증번호,대분류코드,중분류코드,소분류코드,점포코드,구매일자,구매시간,구매금액
0,1,M,60세이상,60.0,강남구,A,1239451,2,201,A020105,12,20140118,15,10000
1,1,M,60세이상,60.0,강남구,A,1319908,4,402,A040214,20,20140203,15,129000
2,1,M,60세이상,60.0,강남구,A,1807440,1,103,A010302,12,20140425,12,7000
3,1,M,60세이상,60.0,강남구,A,1807440,1,104,A010404,12,20140425,12,2650
4,1,M,60세이상,60.0,강남구,A,1807440,1,105,A010501,12,20140425,12,9800


In [21]:
s_df_purchase.shape

(8181992, 14)

In [42]:
# 구매내역 데이터 파일 크기가 크기 때문에 서울시 구매내역으로 데이터를 대체하여 새로 저장한다.
s_df_purchase.to_csv('./data/02_s_purchaseTR.csv', index=False)

In [43]:
# 서울시 고객의 구매내역 데이터 불러오기
s_df_purchase = pd.read_csv('./data/02_s_purchaseTR.csv')
s_df_purchase.head()

,Unnamed: 0,고객번호,성별,연령대,거주지역,S_koo,제휴사,영수증번호,대분류코드,중분류코드,소분류코드,점포코드,구매일자,구매시간,구매금액
0,0,1,M,60세이상,60.0,강남구,A,1239451,2,201,A020105,12,20140118,15,10000
1,1,1,M,60세이상,60.0,강남구,A,1319908,4,402,A040214,20,20140203,15,129000
2,2,1,M,60세이상,60.0,강남구,A,1807440,1,103,A010302,12,20140425,12,7000
3,3,1,M,60세이상,60.0,강남구,A,1807440,1,104,A010404,12,20140425,12,2650
4,4,1,M,60세이상,60.0,강남구,A,1807440,1,105,A010501,12,20140425,12,9800


## 2-3. product data

In [23]:
# 상품정보 불러오기
df_product =  pd.read_csv('./data/03product.csv', encoding='CP949')
df_product.head()

,제휴사,대분류코드,중분류코드,소분류코드,중분류명,소분류명
0,A,1,101,A010101,일용잡화,위생세제
1,A,1,101,A010102,일용잡화,휴지류
2,A,1,101,A010103,일용잡화,뷰티상품
3,A,1,101,A010104,일용잡화,일용잡화
4,A,1,101,A010105,일용잡화,세트상품


## 2-4. channel data

In [27]:
# 채널정보 불러오기
df_channel = pd.read_csv("./data/06channel.csv", encoding="cp949")
df_channel.head()

,고객번호,제휴사,이용횟수
0,7,A_MOBILE/APP,4
1,14,A_MOBILE/APP,1
2,42,B_MOBILE/APP,23
3,74,A_MOBILE/APP,1
4,74,B_MOBILE/APP,30


In [30]:
df_channel.isnull().sum()

고객번호    0
제휴사     0
이용횟수    0
dtype: int64

In [28]:
# 서울시 고객의 채널 데이터 변경
s_df_channel = pd.merge(s_df_client, df_channel, how='left')
s_df_channel.head()

,고객번호,성별,연령대,거주지역,S_koo,제휴사,이용횟수
0,1,M,60세이상,60.0,강남구,NaN,NaN
1,3,M,60세이상,33.0,은평구,NaN,NaN
2,4,F,60세이상,16.0,노원구,NaN,NaN
3,7,F,60세이상,36.0,서대문구,A_MOBILE/APP,4.0
4,8,M,60세이상,10.0,강북구,NaN,NaN


# 3. EDA(Exploratory Data Analysis)

## 2-1. client data

## 2-2. purchase data

## 2-3. product data

## 2-4. channel data

In [36]:
s_df_channel[s_df_channel['고객번호'].duplicated()]

,고객번호,성별,연령대,거주지역,S_koo,제휴사,이용횟수
40,74,F,60세이상,55.0,송파구,B_MOBILE/APP,30.0
41,74,F,60세이상,55.0,송파구,B_ONLINEMALL,11.0
196,376,F,60세이상,24.0,동대문구,C_ONLINEMALL,1.0
216,403,M,60세이상,49.0,광진구,B_ONLINEMALL,13.0
222,418,F,60세이상,39.0,마포구,B_ONLINEMALL,1.0
...,...,...,...,...,...,...,...
6309,18795,F,45세~49세,33.0,은평구,C_ONLINEMALL,15.0
6311,18799,F,20세~24세,33.0,은평구,D_MOBILE/APP,1.0
6312,18799,F,20세~24세,33.0,은평구,B_ONLINEMALL,1.0
6361,18932,F,40세~44세,55.0,송파구,B_MOBILE/APP,1.0


In [32]:
df_channel.shape

(8824, 3)

In [31]:
s_df_channel.isnull().sum()

고객번호        0
성별          0
연령대         0
거주지역        0
S_koo       0
제휴사      4042
이용횟수     4042
dtype: int64

In [ ]:
# 전체 데이터 8824개 중 4042의 서울시 고객만이 모바일이나 웹페이지를 이용하지 않은 것을 확인할 수 있다.
# 총 5841명 중 대다수가 이용하지 않는다는 사실을 알 수 있다.

# 혁진아! null값 지우즈아!!

# 4. Data Preprocessing

# 4-1. Clustering
* 고객의 구매 내역 데이터에서 (구매빈도, 구매시간, 구매금액) 기준으로 clustering 진행
* KMeans, 계층적군집분석, DBSCAN 시도
* KMeans cluster 데이터 분포가 적절하여 해당 방식 이용

In [ ]:
# 구매빈도
df_stuff_cnt = df_purchase['소분류코드'].groupby(df_purchase['소분류코드']).count()
# 구매시간
df_stuff_time = df_purchase['구매시간'].groupby(df_purchase['소분류코드']).mean()
# 구매금액
df_stuff_amt = df_purchase['구매금액'].groupby(df_purchase['소분류코드']).mean()

frame1 = { '구매빈도': df_stuff_cnt, '구매시간': df_stuff_time, '구매금액':df_stuff_amt } 

df_stuff = pd.DataFrame(frame1)
df_stuff.head()

### KMeans

In [ ]:
feature = df_stuff[ ['구매빈도', '구매시간', '구매금액']]

In [ ]:
# 적절한 cluster 수 확인 (but 편차때문에 강제로 늘려버림 5 -> 10개)
ks = range(1,20)
inertias = []

for k in ks:
    model = KMeans(n_clusters=k)
    model.fit(feature)
    inertias.append(model.inertia_)

# Plot ks vs inertias
plt.plot(ks, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

In [ ]:
# 정규표준화, KMeans, 파이프라인 선언
scaler = StandardScaler()
model = KMeans(n_clusters=10)
pipeline = make_pipeline(scaler,model)

In [ ]:
# 파이프라인을 이용하여 정규표준화 한 뒤 KMeans 학습
pipeline.fit(feature)
predict = pd.DataFrame(pipeline.predict(feature))
predict.columns=['predict']

In [ ]:
# clustring 결과의 분포가 적절히 이루어진 것을 확인할 수 있다.
predict['predict'].value_counts()

In [ ]:
# clustring 데이터와 결과값 데이터를 합침
predict = predict.set_index(feature.index)
r = pd.concat([feature,predict],axis=1)
r.head()

In [ ]:
# KMeans 결과값 저장
r.to_csv('./data/r_KMeans.csv')

# 4-2. 

In [ ]:
product_code = df_purchase['소분류코드']
customer_no = df_purchase['고객번호']

frame2 = { '소분류코드': product_code, '고객번호': customer_no} 

pur_product = pd.DataFrame(frame2)
pur_product.head()

In [ ]:
# 구매데이터의 소분류코드, 고객번호와 제품데이터의 clustring 결과값을 merge
pur_prod = pd.merge(pur_product, r['predict'].reset_index(), how='left')
pur_prod.head()

In [ ]:
# cluster에 대한 가변수 생성
cluster_dumm = pd.get_dummies(pur_prod['predict'], prefix='cluster')
cluster_dumm.head()

In [ ]:
purchase_product = pd.concat([pur_prod,cluster_dumm],axis=1)
purchase_product = pd.concat([purchase_product['고객번호'], purchase_product.iloc[:,3:]], axis=1)
purchase_product.head()

In [ ]:
# 고객별 제품cluster 구매 횟수
customer_product_cluster = p_product.groupby(p_product['고객번호']).sum()
customer_product_cluster.head()

In [ ]:
# 고객별 제품cluster csv파일로 저장
customer_product_cluster.to_csv('./data/cust_KMeans.csv')